# modeling13-analysis of decisive data

- Analyzing decisive data

In [ ]:
import numpy as np
import pathlib
from pprint import pprint
import pandas as pd
from util import utils
import re
import os

In [ ]:
gold_input = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR/binary_decisive_format_gold_ctx100id_test.json'

In [ ]:
in_data = utils.open_json(gold_input)

In [ ]:
in_data[0]

for instance in in_data:
#     if instance['gold'] in instance['ctx']['text']:
#         print(f'gold exist in text')
#         print(f"em : {instance['em']}")
#     elif instance['gold'] in instance['ctx']['title']:
#         print(f'gold exist in title')
#         print(f"em : {instance['em']}")
        
#     else:
#         print(f'gold not exist')
#         print(f"em : {instance['em']}")
    if instance['em'] == '0':
        pprint(instance)
        input("Press Enter to continue...")

In [ ]:
len(in_data)

    {'answers': ['middle layer', 'The uvea', 'uvea'],
     'ctx': {'id': '2829741',
             'text': 'Choroid The choroid, also known as the choroidea or choroid '
                     'coat, is the vascular layer of the eye, containing '
                     'connective tissues, and lying between the retina and the '
                     'sclera. The human choroid is thickest at the far extreme '
                     'rear of the eye (at 0.2 mm), while in the outlying areas it '
                     'narrows to 0.1 mm. The choroid provides oxygen and '
                     'nourishment to the outer layers of the retina. Along with '
                     'the ciliary body and iris, the choroid forms the uveal '
                     'tract. The structure of the choroid is generally divided '
                     'into four layers (classified in order of furthest away from '
                     'the',
             'title': 'Choroid'},
     'em': '0',
     'gold': 'uvea',
     'id': 56,
     'inference': 'choroid',
     'question': 'the vascular layer of the eye is the'}

## Incremental Testing Setting

### 3 Method (Check out README.md)

### Method 2 (2nd method of in README.md)
    - The sooner, The better approach
      - Keep the Positive Context in order
    - Since we testing it, let's keep sample size of 5
    - When there is no Exact Mathcing during the incremental inference e.g.) em_pattern = '00000'
        - Keep the whole context so that those cases will have False on Exact Match values
        
    - Patterns: 
        - first 1 : positive
        - first 0 : irrelevant 
            - we know that first is the positive context that includes answer
            - with that perspective, it is relevant in theory
            - ; however, dataset is created via BLEU score on two different corpus.
            - If FiD does not correctly infer the output, 
            - context what we concieved of artificial positive context is actually irrelevant to the query.
            - Also when realistic scenario, we don't know whether the first context contains the answer
        - 01 pattern : Positive 
        - 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
        - 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
        - 10 pattern : Damaging
        
        - 11 pattern : relevant vs positive
            Strict Positive
                Under strict rule : consider it as relevant ?(or irrelevant?)
                                in terms of strictly limiting the number of positive passages
            Naive Positive
                Under naive rule : consider it as positive
                                in that this would increase the number of positive passages
                
        - A00 pattern : irrelevant vs damaging
            if '1' does not occured in A, currnet passage is irrelevant
            if '1' occurred in A, current passage is damaging either irrelevant 
                Strict Damaging
                    Under strict rule : consider it as irrelevnat 
                                    in terms of strictly limiting the number of damaging passage
                Naive Damaging
                    Under naive rule : consider it as damaging 
                                    in that this would increase the number of damaging passages
        
    - Options1 : Removes only damaging
    - Options2 : Removes damaging + irrelevant
    - Options3 : Removes damaging + relevant
    - Options4 : Removes damaging + irrelevant + relevnat
        

# Input
    - result from 5-1
    - /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json

# Trials
    - Need to check FiD input when there are less ctxs than n_context

# Binary Classification Output File

In [ ]:
sample_size = 100
# input_file = f'/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-adpative-decisive-trial7/step_360/intact_prediction/prediction.json'
# input_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-linear-decisive-trial6/step_80/intact_prediction'
input_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_data-weighted-trial3_desktop1/step_240/intact_prediction'

input_file = 'prediction.json'
input_file_path = os.path.join(input_path, input_file)
input_data = utils.open_json(input_file_path)
pprint((input_data[5]))


In [ ]:
len(input_data)

In [ ]:
predict_id = {}
for ins in input_data:
    if ins['id'] not in predict_id:
        predict_id[ins['id']] = str(ins['binary_inference'])
    else:
        predict_id[ins['id']] += str(ins['binary_inference'])

In [ ]:
print(len(predict_id))

In [ ]:
print(len(predict_id[0]))

# Binary Classification Output File -> DPR format

In [ ]:
dpr_test_json = '/data/philhoon-relevance/binary-classification/NQ-TEST-DPR/ctx100id.json'

In [ ]:
dpr_test_data = utils.open_json(dpr_test_json)
print(dpr_test_data[0].keys())

In [ ]:
for ins in dpr_test_data:
    prediction_em_pattern = predict_id[ins['id']]
    ins['prediction_em_pattern'] = prediction_em_pattern
    

In [ ]:
print(dpr_test_data[1].keys())

In [ ]:
input_path

In [ ]:
output_path = input_path
output_file_name = 'ctx100id_prediction_em.json'
output_file_path = os.path.join(output_path, output_file_name)
print(output_file_path)

In [ ]:
utils.save_json(dpr_test_data, output_file_path)

# Method Selection

In [ ]:
def build_data_on_prediction(input_file, option, option_p, option_d, sample_size):
    '''
    input_file : incremental inference result from FiD from KILT-5-1
        path : /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json
        
    output : FiD input json format
    
    option(required) : removing strategies
        op1 : removes damages only
        op2 : removes damaging + irrelevant
        op3 : removes damaging + relevant
        op4 : removes damaging + irrelevant + relevant
        
    option_p(required) : positive passage selection options
        strict : strict positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is relevant
        naive : naive positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is positive
                
    option_d(required) : damaging passage selection options
        strict : strict negative
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is irrelevant
        naive : naive damaging
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is damaging
    
    '''
    
    output_format = []
    null_em = '0' * sample_size
    
    # 'strict', 'naive'  
    # option_p = 'naive'
    # option_d = 'naive'
    # option = 'op4'

    for id_, instance in enumerate(input_file,1):
        template_dict = {}
        if 'id' in instance.keys():
            template_dict['id'] = instance['id']
        else:
            template_dict['id'] = str(id_)
        template_dict['answers'] = instance['answers']
        template_dict['question'] = instance['question']
#         template_dict['em_pattern'] = instance['em_pattern']
        template_dict['em_pattern'] = instance['em_pattern']
        template_dict['prediction_em_pattern'] = instance['prediction_em_pattern']

        em_pattern = instance['prediction_em_pattern']

        # when there is at least one EM in the accumulated inference
        if em_pattern != null_em:   
            new_ctx = []

            # relevant vs positive
            positve_ctx_lst = []
            relevant_ctx_lst = []

            # irrelevant vs damaging
            damaging_ctx_lst = []
            irrelevant_ctx_lst = []


            for idx_, ctx in enumerate(instance['ctxs']):

                # checking current em
                cur_em = em_pattern[idx_]
                pre_em_pattern = em_pattern[:idx_]


                # first 1 : positive
                if not pre_em_pattern and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # first 0 : irrelevant
                elif not pre_em_pattern and cur_em == '0':
                    irrelevant_ctx_lst.append(ctx)
                    
                # 01 pattern : positive 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # 10 pattern : damaging
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '0':
                    damaging_ctx_lst.append(ctx)

                # 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '1':
                    if option_p == 'strict':
                        relevant_ctx_lst.append(ctx)

                    elif option_p == 'naive':
                        positve_ctx_lst.append(ctx)

                    else:
                        print('option_p should be either \'strict\' or \'naive\'')
                        return 

                # 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '0':
                    # if '1' does not occured in A, currnet passage is irrelevant
                    if not '1' in pre_em_pattern:
                        irrelevant_ctx_lst.append(ctx)

                    # if '1' occurred in A, 
                    else:
                        # strict : consider it as irrelevnat 
                        if option_d == 'strict':
                            irrelevant_ctx_lst.append(ctx)

                        # naive : consider it as damaging 
                        elif option_d == 'naive':
                            damaging_ctx_lst.append(ctx)

                        else:
                            print('option_p should be either \'strict\' or \'naive\'')
                            return 

            # op1 removes damages only
            if option == 'op1':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)


            # op2 removes damaging + irrelevant
            elif option == 'op2':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)

            # op3 : Removes damaging + relevant
            elif option == 'op3':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)

            # op4 : Removes damaging + irrelevant + relevant
            elif option == 'op4':
                new_ctx.extend(positve_ctx_lst)

            else:
                print('option should be op1, op2, op3, op4')
                return 

            template_dict['ctxs'] = new_ctx
            output_format.append(template_dict)

        # when there is no EM in the accumulated inference
        else:
            template_dict['ctxs']= instance['ctxs']
            output_format.append(template_dict)
    
    print('==============instance finished======================')
    return output_format
        


In [ ]:
# option_dict = {
#     'op1' : 'remove_damage',
#     'op2' : 'remove_damage_irrelevant',
#     'op3' : 'remove_damage_relevant',
#     'op4' : 'remove_damage_irrelevant_relevant',
# }
# option = 'op4'

In [ ]:
# op1, op2 are duplicates
method_option_dict = {
    'op3' : 'remove_damage_relevant',
    'op4' : 'remove_damage_irrelevant_relevant',
}

In [ ]:
option_p_dict = {
    'strict' : 'strict_positive',
    'naive' : 'naive_positive',
}
# option_p = 'strict'

In [ ]:
option_d_dict = {
    'strict' : 'strict_damaging',
    'naive' : 'naive_damaging',
}
# option_d = 'strict'

In [ ]:
# input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')
# output_path = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25_SELECTION'

# TEST set Inference

In [ ]:
input_file = 'ctx100id_prediction_em.json'
input_file_path = os.path.join(input_path, input_file)
print(input_file_path)


In [ ]:
input_ = utils.open_json(input_file_path)

In [ ]:
len(input_)

In [ ]:
output_path = input_path
output_path_dir = os.path.join(input_path, 'selection_methods')
print(output_path_dir)

In [ ]:
os.makedirs(output_path_dir, exist_ok = True)

In [ ]:
output_path = output_path_dir

In [ ]:
print(output_path)

In [ ]:
for o_ in method_option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            if option == 'op4' and option_p == 'strict' and option_d == 'strict':
                continue
            if option == 'op4' and option_p == 'naive' and option_d == 'strict':
                continue 
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{method_option_dict[option]}.json'

            output_file = os.path.join(output_path, filename)
#             print(output_file)
            output_format = build_data_on_prediction(input_, option, option_p, option_d, sample_size)
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

- Method3. '00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  (__0.73651__)     

- Method4. '00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>1000 (_0.73611_)    

- Method1. '00011011000'  -> 000<span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span><span style="color:red">1</span>000 (0.71130)    

- Method2. '00011011000'  -> 000<span style="color:red">1</span>10<span style="color:red">1</span>1000  (0.70972)        
- Method6. '00011011000'  -> <span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>10<span style="color:blue">0</span><span style="color:blue">0</span> (0.68609)    

- Method5. '00011011000'  -> <span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:blue">0</span><span style="color:blue">0</span>    (0.68452) 

In [ ]:
method_dict = {
'strict_positive_strict_damaging_remove_damage_relevant.json' : 'Method6', 
'strict_positive_naive_damaging_remove_damage_relevant.json' : 'Method4', 
'naive_positive_strict_damaging_remove_damage_relevant.json' : 'Method5', 
'naive_positive_naive_damaging_remove_damage_relevant.json' : 'Method3', 
'strict_positive_naive_damaging_remove_damage_irrelevant_relevant.json' : 'Method2', 
'naive_positive_naive_damaging_remove_damage_irrelevant_relevant.json' : 'Method1',
}

# Confirm the result

### Method 1 Test
- Method1. '00011011000'  -> 000<span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span><span style="color:red">1</span>000 (0.71130)    


In [ ]:
test_file = 'naive_positive_naive_damaging_remove_damage_irrelevant_relevant.json'
test_input_path = os.path.join(output_path, test_file)
print(test_input_path)
test_data = utils.open_json(test_input_path)


for instance in test_data:
    check_len = [1 if i == '1' else 0 for i in instance['prediction_em_pattern']]
    
    if sum(check_len) != len(instance['ctxs']):
        print('method 1 failed')
        break
else:
    print('pass')

### Method 2 Test
- Method2. '00011011000'  -> 000<span style="color:red">1</span>10<span style="color:red">1</span>1000  (0.70972)     

In [ ]:
def get_definite_pos_neg(test_em):
    positive_pos = []
    if test_em.startswith('1'):
        positive_pos.append(0)
    iter_ = re.finditer(r'01', test_em)
    for m in iter_:
        pos_ = m.start() + 1
        positive_pos.append(pos_)

    negative_pos = []
    iter_ = re.finditer(r'10', test_em)
    for m in iter_:
        pos_ = m.start() + 1
        negative_pos.append(pos_)
    
    return positive_pos, negative_pos

In [ ]:
test_file = 'strict_positive_naive_damaging_remove_damage_irrelevant_relevant.json'
test_input_path = os.path.join(output_path, test_file)
print(test_input_path)
test_data = utils.open_json(test_input_path)


for instance in test_data:
    positive_pos, _ = get_definite_pos_neg(instance['prediction_em_pattern'])
    
    if len(positive_pos) != len(instance['ctxs']):
        print('method 2 failed')
        break
        
else:
    print('pass')

### Method 3 Test
- Method3. '00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  (__0.73651__)   

In [ ]:
test_file = 'naive_positive_naive_damaging_remove_damage_relevant.json'
test_input_path = os.path.join(output_path, test_file)
print(test_input_path)
test_data = utils.open_json(test_input_path)


for instance in test_data:
    prediction_em_pattern = instance['prediction_em_pattern']
    
    check_len = sum([1 if i == '1' else 0 for i in prediction_em_pattern])
    num_undecisive = len(prediction_em_pattern) - len(prediction_em_pattern.lstrip('0'))
    
    
    if check_len + num_undecisive != len(instance['ctxs']):
        print('method 3 failed')
        break
else:
    print('pass')

### Method 4 Test
- Method4. '00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>1000 (_0.73611_)    


In [ ]:
test_file = 'strict_positive_naive_damaging_remove_damage_relevant.json'
test_input_path = os.path.join(output_path, test_file)
print(test_input_path)
test_data = utils.open_json(test_input_path)


for instance in test_data:
    prediction_em_pattern = instance['prediction_em_pattern']
    num_undecisive = len(prediction_em_pattern) - len(prediction_em_pattern.lstrip('0'))
    positive_pos, _ = get_definite_pos_neg(prediction_em_pattern)
    
    if num_undecisive + len(positive_pos) != len(instance['ctxs']):
        print('method 4 failed')
        break
        
else:
    print('pass')

### Method 5 Test
- Method5. '00011011000'  -> <span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:blue">0</span><span style="color:blue">0</span>    (0.68452) 

In [ ]:
test_file = 'naive_positive_strict_damaging_remove_damage_relevant.json'
test_input_path = os.path.join(output_path, test_file)
print(test_input_path)
test_data = utils.open_json(test_input_path)


for instance in test_data:
    prediction_em_pattern = instance['prediction_em_pattern']
    _, negative_pos = get_definite_pos_neg(prediction_em_pattern)
    
    if len(negative_pos) + len(instance['ctxs']) != 100:
        print(f"id : {instance['id']}")
        print(f"p_em_pattern : {instance['prediction_em_pattern']}")
        print(f"em_pattern   : {instance['em_pattern']}")
        print(f'len(negative_pos) : {len(negative_pos)}')
        print(f"len(instance['ctxs'] : {len(instance['ctxs'])}")
                                            
        print('method 5 failed')
        break
        
else:
    print('pass')

### Method 6 Test
- Method6. '00011011000'  -> <span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>10<span style="color:blue">0</span><span style="color:blue">0</span> (0.68609)    




In [ ]:
test_file = 'strict_positive_strict_damaging_remove_damage_relevant.json'
test_input_path = os.path.join(output_path, test_file)
print(test_input_path)
test_data = utils.open_json(test_input_path)


for instance in test_data:
    prediction_em_pattern = instance['prediction_em_pattern']
    num_undecisive = len(prediction_em_pattern) - len(prediction_em_pattern.lstrip('0'))
    positive_pos, negative_pos = get_definite_pos_neg(prediction_em_pattern)

    check_0 = sum([1 if i == '0' else 0 for i in instance['prediction_em_pattern']])
    rest_0 = check_0 - len(negative_pos) - num_undecisive
    
    included = num_undecisive + len(positive_pos) + rest_0

    if included != len(instance['ctxs']):
        print('method 6 failed')
        break
        
else:
    print('pass')

## NQ dev set

In [ ]:
sample_size = 100
input_file = f'/data/philhoon-relevance/FiD/results/NQ_DPR/DEV/incremental_result_{sample_size}/ctx{sample_size}.json'
output_path = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_DPR_DEV_SELECTION/ctx_{sample_size}'

In [ ]:
input_ = utils.open_json(input_file)

In [ ]:
output_path

In [ ]:
len(input_)

In [ ]:
for o_ in method_option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            if option == 'op4' and option_p == 'strict' and option_d == 'strict':
                continue
            if option == 'op4' and option_p == 'naive' and option_d == 'strict':
                continue 
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{method_option_dict[option]}.json'

            output_file = os.path.join(output_path, filename)
            output_format = build_data(input_, option, option_p, option_d, sample_size)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

# Method explanation
- Redundancies in selection strategies -> remove them -> 6

#### Method1. Include passages that corresponds to 1s
- all __option2__ belong to this case
- option4_naive_naive, option4_naive_strict

'00011011000'  -> 000<span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span><span style="color:red">1</span>000  
- red 1s

#### Method2. Include passages that corresponds to first 1s
- option4_strict_naive, option4_strict_strcit 

'00011011000'  -> 
000<span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s
- removing consecutive passages when the previous output is correct as well

#### Method3. Include passages that corresponds to 1s + Included First appeared consecutive 0s
- option1_naive_naive, option1_strict_naive
- option3_naive_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  
- red 1s and blue 0s
- usually top-retrieved results contain the answer 

#### Method4. Include passages that corresponds to first 1s + Included First appeared consecutive 0s
- option3_strict_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- usually top-retrieved results contain the answer 

#### Method5. Include passages that corresponds to 1s + Remove only damaging
- option1_naive_strict, option1_strict_strict
- option3_naive_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:blue">0</span><span style="color:blue"><span style="color:blue">0</span><span style="color:blue"> 
- red 1s and blue 0s
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

#### Method6. Include passages that corresponds to first 1s  + Remove only damaging
- option3_strict_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>10<span style="color:blue">0</span><span style="color:blue">0</span>
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

## Conclusion
- option1, option2 are not needed
- Method1 : option4_naive_naive
- Method2 : option4_strict_naive 
- Method3 : option3_naive_naive
- Method4 : option3_strict_naive
- Method5 : option3_naive_strict
- Method6 : option3_strict_strict